In [8]:
import string
from random import shuffle
import autograd.numpy as np
from autograd import grad
from nltk.corpus import stopwords
import operator
import nltk

In [9]:
## Calculate the total number of class and words for the training data file
stemmer = nltk.wordnet.WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

class_train = [] ## store all the speaker in the training file 
word_train = [] ## store all the words in the training file

with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/train') as f_train:
    for line in f_train:
        ## remove all the punctuations and stopwords
        line_lt = [w for w in line.split() if w not in string.punctuation if w not in stop_words] 
        ## remove all the affixes
        line_list = []
        line_list.append(line_lt[0])
        for item in line_lt[1:]:
            line_list.append(stemmer.lemmatize(item))   ## use Wordnet Stemmer to remove affix of each word except for the speaker name
        
        class_train.append(line_list[0])
        
        for i in line_list[1:]:
            word_train.append(i)
            
all_class_train = list(set(class_train))
all_word_train = list(set(word_train))

all_class_train_sorted = sorted (all_class_train)
all_word_train_sorted = sorted(all_word_train)

print (len(all_class_train))
print (len(all_word_train))

17
8146


In [10]:
## Define a function to calculate -log(P(k|d))

def negative_log_likelihood(lamda,document): # lamda is row vector contain all the parameters
    word_lt = [w for w in document.split() if w not in string.punctuation if w not in stop_words]
    word_list = []
    word_list.append(word_lt[0])
    for i in word_lt[1:]:
        word_list.append(stemmer.lemmatize(i))
    
    word_count = {}
    for i in word_list[1:]:
        if i not in word_count:
            word_count[i] = 1
        else:
            word_count[i] += 1
            
    ## construct word count array
    word_count_list = []
    word_count_list.append(1)
    for i in all_word_train_sorted:
        if i in word_count:
            word_count_list.append(word_count[i])
        else:
            word_count_list.append(0)
            
    ## calculate the negative_log_likelihood
    s_logistic = {}
    for i in all_class_train_sorted:
        inde = all_class_train_sorted.index(i)
        length = len(word_count_list)
        s_logistic[i] = lamda[inde*length:(inde*length+length)] @ np.array([word_count_list]).T
        
    max_s_logistic = max(s_logistic.values())
    for i in s_logistic:
        s_logistic[i] = s_logistic[i] - max_s_logistic ## deal with the under-flow problem
        
    target_class = word_list[0]
    
    numerator = np.exp(s_logistic[target_class])
    denome = 0 #What if use None?
    for i in s_logistic:
        denome += np.exp(s_logistic[i])
    
    ## Add l2 regularization smoothing
    #C=0.1 ## regularization parameter
    
    return -np.log(numerator/denome)
    #return -np.log((C/2)*np.linalg.norm(lamda)+numerator/denome)
    

In [11]:
## define the gradient function
gradient_negative_log_likelihood = grad(negative_log_likelihood)

In [12]:
## define prediction function
def prediction_class(lamda,document): ## if predict correctly, return 1; else return 0
    word_lt = [w for w in document.split() if w not in string.punctuation if w not in stop_words]
    word_list = []
    word_list.append(word_lt[0])
    for i in word_lt[1:]:
        word_list.append(stemmer.lemmatize(i))
        
    word_count = {}
    for i in word_list[1:]:
        if i not in word_count:
            word_count[i] = 1
        else:
            word_count[i] += 1
            
    ## construct word count array,just ignore the unseen words
    word_count_list = []
    word_count_list.append(1)
    for i in all_word_train_sorted:
        if i in word_count:
            word_count_list.append(word_count[i])
        else:
            word_count_list.append(0)
            
    ## calculate the negative_log_likelihood
    s_logistic = {}
    for i in all_class_train_sorted:
        inde = all_class_train_sorted.index(i)
        length = len(word_count_list)
        s_logistic[i] = lamda[inde*length:(inde*length+length)] @ np.array([word_count_list]).T
        
    max_s_logistic = max(s_logistic.values())
    for i in s_logistic:
        s_logistic[i] = s_logistic[i] - max_s_logistic
        
    
    prediction_prb = {}
    
    denome = 0
    for i in s_logistic:
        denome += np.exp(s_logistic[i])
        
    for i in s_logistic:
        prediction_prb[i] = np.exp(s_logistic[i])/denome
    
    
    predict_speaker = [key for key,val in prediction_prb.items() if val == max(prediction_prb.values())]
    target_class = word_list[0]
    
    if predict_speaker[0] == target_class:
        return 1
    else:
        return 0  

In [13]:
## Put all the training document in a list
all_document_train = []
with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/train') as f_train:
    for document in f_train: 
        all_document_train.append(document)

In [14]:
## Learning rate is 0.001
lamda1 = np.zeros(len(all_class_train)*len(all_word_train)+len(all_class_train))
yi = 0.001 ## learning rate

for i in range(40):
    print ("Iteration {}".format(i+1))
    shuffle(all_document_train)  ## Add random shuffling of the documents
    sum_alldoc_negative_log_likelihood = 0
    for document in all_document_train:
        lamda1 = lamda1 - yi*gradient_negative_log_likelihood(lamda1,document)
        sum_alldoc_negative_log_likelihood += negative_log_likelihood(lamda1,document)
    print ("-log(P(k|d)): {}".format(sum_alldoc_negative_log_likelihood))
    with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/dev') as f_dev:
        count_documnet=0
        count_correct_prediction=0
        for document in f_dev:
            count_documnet+=1
            count_correct_prediction=count_correct_prediction+prediction_class(lamda1,document)
    print ('Precition accuracy of dev file is {}'.format(count_correct_prediction/count_documnet))                    

Iteration 1
-log(P(k|d)): [ 7979.1005168]
Precition accuracy of dev file is 0.3525
Iteration 2
-log(P(k|d)): [ 7072.65884008]
Precition accuracy of dev file is 0.3825
Iteration 3
-log(P(k|d)): [ 6542.85813816]
Precition accuracy of dev file is 0.3975
Iteration 4
-log(P(k|d)): [ 6164.53983566]
Precition accuracy of dev file is 0.42
Iteration 5
-log(P(k|d)): [ 5875.96984476]
Precition accuracy of dev file is 0.43
Iteration 6
-log(P(k|d)): [ 5649.269923]
Precition accuracy of dev file is 0.445
Iteration 7
-log(P(k|d)): [ 5464.117579]
Precition accuracy of dev file is 0.4525
Iteration 8
-log(P(k|d)): [ 5308.85479086]
Precition accuracy of dev file is 0.465
Iteration 9
-log(P(k|d)): [ 5176.27649971]
Precition accuracy of dev file is 0.4625
Iteration 10
-log(P(k|d)): [ 5060.76671784]
Precition accuracy of dev file is 0.4625
Iteration 11
-log(P(k|d)): [ 4958.68124019]
Precition accuracy of dev file is 0.465
Iteration 12
-log(P(k|d)): [ 4867.08760839]
Precition accuracy of dev file is 0.4675
I

In [15]:
## Learning rate is 0.01
lamda2 = np.zeros(len(all_class_train)*len(all_word_train)+len(all_class_train))
yi = 0.01 ## learning rate

for i in range(40):
    print ("Iteration {}".format(i+1))
    shuffle(all_document_train)  ## Add random shuffling of the documents
    sum_alldoc_negative_log_likelihood = 0
    for document in all_document_train:
        lamda2 = lamda2 - yi*gradient_negative_log_likelihood(lamda2,document)
        sum_alldoc_negative_log_likelihood += negative_log_likelihood(lamda2,document)
    print ("-log(P(k|d)): {}".format(sum_alldoc_negative_log_likelihood))
    with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/dev') as f_dev:
        count_documnet=0
        count_correct_prediction=0
        for document in f_dev:
            count_documnet+=1
            count_correct_prediction=count_correct_prediction+prediction_class(lamda2,document)
    print ('Precition accuracy of dev file is {}'.format(count_correct_prediction/count_documnet))

Iteration 1
-log(P(k|d)): [ 5444.67112549]
Precition accuracy of dev file is 0.4725
Iteration 2
-log(P(k|d)): [ 4343.29715592]
Precition accuracy of dev file is 0.475
Iteration 3
-log(P(k|d)): [ 3959.01639208]
Precition accuracy of dev file is 0.495
Iteration 4
-log(P(k|d)): [ 3715.71841679]
Precition accuracy of dev file is 0.5025
Iteration 5
-log(P(k|d)): [ 3538.13616818]
Precition accuracy of dev file is 0.505
Iteration 6
-log(P(k|d)): [ 3400.12197156]
Precition accuracy of dev file is 0.505
Iteration 7
-log(P(k|d)): [ 3282.52204026]
Precition accuracy of dev file is 0.51
Iteration 8
-log(P(k|d)): [ 3184.09862853]
Precition accuracy of dev file is 0.515
Iteration 9
-log(P(k|d)): [ 3096.2638701]
Precition accuracy of dev file is 0.5075
Iteration 10
-log(P(k|d)): [ 3018.36504115]
Precition accuracy of dev file is 0.5075
Iteration 11
-log(P(k|d)): [ 2948.70159175]
Precition accuracy of dev file is 0.505
Iteration 12
-log(P(k|d)): [ 2885.0993178]
Precition accuracy of dev file is 0.52
I

In [16]:
## Learning rate is 0.1
lamda3 = np.zeros(len(all_class_train)*len(all_word_train)+len(all_class_train))
yi = 0.1 ## learning rate

for i in range(40):
    print ("Iteration {}".format(i+1))
    shuffle(all_document_train)  ## Add random shuffling of the documents
    sum_alldoc_negative_log_likelihood = 0
    for document in all_document_train:
        lamda3 = lamda3 - yi*gradient_negative_log_likelihood(lamda3,document)
        sum_alldoc_negative_log_likelihood += negative_log_likelihood(lamda3,document)
    print ("-log(P(k|d)): {}".format(sum_alldoc_negative_log_likelihood))
    with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/dev') as f_dev:
        count_documnet=0
        count_correct_prediction=0
        for document in f_dev:
            count_documnet+=1
            count_correct_prediction=count_correct_prediction+prediction_class(lamda3,document)
    print ('Precition accuracy of dev file is {}'.format(count_correct_prediction/count_documnet))

Iteration 1
-log(P(k|d)): [ 2813.37283508]
Precition accuracy of dev file is 0.5025
Iteration 2
-log(P(k|d)): [ 2285.6850709]
Precition accuracy of dev file is 0.5075
Iteration 3
-log(P(k|d)): [ 1999.75679229]
Precition accuracy of dev file is 0.525
Iteration 4
-log(P(k|d)): [ 1798.48731192]
Precition accuracy of dev file is 0.515
Iteration 5
-log(P(k|d)): [ 1658.98739382]
Precition accuracy of dev file is 0.52
Iteration 6
-log(P(k|d)): [ 1554.69326447]
Precition accuracy of dev file is 0.5075
Iteration 7
-log(P(k|d)): [ 1464.38351193]
Precition accuracy of dev file is 0.4975
Iteration 8
-log(P(k|d)): [ 1391.07077953]
Precition accuracy of dev file is 0.4975
Iteration 9
-log(P(k|d)): [ 1332.60228668]
Precition accuracy of dev file is 0.5025
Iteration 10
-log(P(k|d)): [ 1276.30785065]
Precition accuracy of dev file is 0.505
Iteration 11
-log(P(k|d)): [ 1229.02084947]
Precition accuracy of dev file is 0.52
Iteration 12
-log(P(k|d)): [ 1187.91391527]
Precition accuracy of dev file is 0.51

In [31]:
results_lamda = {}
results_lamda ['learning_rate:0.001, iteration: 40'] = lamda1
results_lamda ['learning_rate:0.01, iteration: 40'] = lamda2
results_lamda ['learning_rate:0.1, iteration: 40'] = lamda3

In [22]:
prediction_accuracy_test = {}

In [27]:
## Calculate the prediction accuracy of test file

with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/test') as f_test:
    count_documnet=0
    count_correct_prediction=0
    for document in f_test:
        count_documnet+=1
        count_correct_prediction=count_correct_prediction+prediction_class(results_lamda ['learning_rate:0.001, iteration: 40'],document)
#print ('Precition accuracy of test file is {}'.format(count_correct_prediction/count_documnet))
prediction_accuracy_test['learning_rate:0.001, iteration: 40'] = count_correct_prediction/count_documnet

In [28]:
## Print out the prediction accuracy on test file
print (prediction_accuracy_test)
## Comments: when the learning rate is 0.01, iteration is 40, the trained parameters can give the best predicition on test file

{'learning_rate:0.01, iteration: 40': 0.535, 'learning_rate:0.001, iteration: 40': 0.5125, 'learning_rate:0.1, iteration: 40': 0.4875}


In [32]:
## Print out the intersted parameters

## Caculate λ(k) and λ(k,w) for k∈{clinton,trump} and w∈{country,president}.
index_clintion = all_class_train_sorted.index('clinton')
index_trump = all_class_train_sorted.index('trump')
index_country = all_word_train_sorted.index('country')
index_president = all_word_train_sorted.index('president')

## I store all the lambda values in a np.array. So I need to find the index of the targetted lambda

length = len(all_word_train_sorted)

lamda_clinton = results_lamda ['learning_rate:0.01, iteration: 40'][index_clintion*(length+1)]
lamda_trump = results_lamda ['learning_rate:0.01, iteration: 40'][index_trump*(length+1)]
lamda_clinton_country = results_lamda ['learning_rate:0.01, iteration: 40'][index_clintion*(length+1)+(index_country+1)]
lamda_clinton_president = results_lamda ['learning_rate:0.01, iteration: 40'][index_clintion*(length+1)+(index_president+1)]
lamda_trump_country = results_lamda ['learning_rate:0.01, iteration: 40'][index_trump*(length+1)+(index_country+1)]
lamda_trump_president = results_lamda ['learning_rate:0.01, iteration: 40'][index_trump*(length+1)+(index_president+1)]

print ("λ(clinton): {}".format(lamda_clinton))
print ("λ(trump): {}".format(lamda_trump))
print ("λ(clinton,country): {}".format(lamda_clinton_country))
print ("λ(clinton,president): {}".format(lamda_clinton_president))
print ("λ(trump,country): {}".format(lamda_trump_country))
print ("λ(trump,president): {}".format(lamda_trump_president))

λ(clinton): 1.2723878597764715
λ(trump): 2.065356710560351
λ(clinton,country): -0.36289273471693506
λ(clinton,president): 0.6444733976895954
λ(trump,country): 0.4054652195830286
λ(trump,president): -0.5676129289825407


In [39]:
## define prediction function -- print out the predicted probability for each class
def prediction_class_print_prob_eachclass(lamda,document): ## if predict correctly, return 1; else return 0
    word_lt = [w for w in document.split() if w not in string.punctuation if w not in stop_words]
    word_list = []
    word_list.append(word_lt[0])
    for i in word_lt[1:]:
        word_list.append(stemmer.lemmatize(i))
        
    word_count = {}
    for i in word_list[1:]:
        if i not in word_count:
            word_count[i] = 1
        else:
            word_count[i] += 1
            
    ## construct word count array,just ignore the unseen words
    word_count_list = []
    word_count_list.append(1)
    for i in all_word_train_sorted:
        if i in word_count:
            word_count_list.append(word_count[i])
        else:
            word_count_list.append(0)
            
    ## calculate the negative_log_likelihood
    s_logistic = {}
    for i in all_class_train_sorted:
        inde = all_class_train_sorted.index(i)
        length = len(word_count_list)
        s_logistic[i] = lamda[inde*length:(inde*length+length)] @ np.array([word_count_list]).T
        
    max_s_logistic = max(s_logistic.values())
    for i in s_logistic:
        s_logistic[i] = s_logistic[i] - max_s_logistic
        
    
    prediction_prb = {}
    
    denome = 0
    for i in s_logistic:
        denome += np.exp(s_logistic[i])
        
    for i in s_logistic:
        prediction_prb[i] = np.exp(s_logistic[i])/denome
    
    print(prediction_prb) 
    
    sum_allpro = 0
    for i in prediction_prb:
        sum_allpro += prediction_prb[i]
    
    print ("sum of the predicted probability of each class: {}".format(sum_allpro))
    predict_speaker = [key for key,val in prediction_prb.items() if val == max(prediction_prb.values())]
    target_class = word_list[0]
    
    print("Predicted speaker: {}".format(predict_speaker))
    print("True speaker: {}".format(target_class))

In [40]:
## print out the predicted probability for each class of the first document in dev file 
with open ('C:/Users/Xin/Desktop/NLP/Homework1/hw1-data/dev') as f_dev:
    prediction_class_print_prob_eachclass(results_lamda ['learning_rate:0.01, iteration: 40'],next(f_dev))

{'trump': array([ 0.00088546]), 'fiorina': array([ 0.0044735]), "o'malley": array([ 0.00454342]), 'cruz': array([ 0.00259016]), 'walker': array([ 0.00246246]), 'rubio': array([ 0.11651084]), 'paul': array([ 0.01190163]), 'kasich': array([ 0.00354336]), 'clinton': array([ 0.76896159]), 'christie': array([ 0.05027694]), 'huckabee': array([ 0.0017373]), 'perry': array([ 0.00013387]), 'bush': array([ 0.01094699]), 'carson': array([ 0.00467844]), 'sanders': array([ 0.00424586]), 'webb': array([ 0.01144022]), 'chafee': array([ 0.00066796])}
sum of the predicted probability of each class: [ 1.]
Predicted speaker: ['clinton']
True speaker: webb
